# Import Libraries

In [ ]:
#System Libraries
import time
from datetime import datetime

#DataFrames manipulation
import pandas as pd

#Connect and Download Data from websites
from selenium import webdriver
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Download Annual IVV Holdings

## Download data from 2019 - 2022 (EDGAR NPORT-P)

Extra Adjustments on the data:
- Delete the holdings relate to Cash and pro-rate the weights
- Sum the weight of the duplicated values (because there are stocks with different asset classes - e.g. Alphabet/Google)

In [ ]:
#Import the file with the links to IVV NPORT-P from EDGAR
edgar_portp_links = pd.read_csv('EDGAR PORT-P links.csv')

#Get the years available
years = edgar_portp_links['Date'].to_list()

In [ ]:
#Create a writer object to save the dataframe to an excel file
writer = pd.ExcelWriter('IVV_EDGAR_holdings.xlsx', engine='xlsxwriter')

#Loop through each year
for year in years:
    #Get the Link for each year
    link = edgar_portp_links[edgar_portp_links['Date'] == year]['HTML'].iloc[0]
    
    driver = webdriver.Chrome(executable_path=r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')  # Optional argument, if not specified will search path.
    driver.get(link)

    time.sleep(2)
    
    #Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    #Find all the invstOrSec elements
    invstOrSecs = soup.find_all('invstorsec')

    #Create an empty list to store the extracted data
    data = []

    #Loop through each invstOrSec element
    for invstOrSec in invstOrSecs:
        try:
            name = invstOrSec.find('name').text
            title = invstOrSec.find('title').text
            isin = invstOrSec.find('isin').get('value')
            pctVal = invstOrSec.find('pctval').text

            # Add the data to the list
            data.append([name, title, isin, pctVal])
        except:
            continue
            
    #Create a pandas DataFrame from the data list
    df = pd.DataFrame(data, columns=['Name', 'Title', 'ISIN', 'Percentage Value'])
    


    #Save the dataframe to a different tab in the excel file
    df.to_excel(writer, sheet_name = str(year))
   
    driver.close()
    driver.quit()
    
#Save and close the writer object
writer.save()

## Download data from 2005 - 2018 (EDGAR N-Q)

Need to download the data mannually as there are more ETFs included in the N-Q files
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=C000012040

Extra Adjustments on the data:
- Donwload only the stocks and calculate the weights accordingly 
- Sum the weight of the duplicated values (because there are stocks with different asset classes - e.g. Alphabet/Google)

# XLG vs. IVV

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('IVV XLG')
df = tickers.history(start = '2022-12-30')

#Reorganize the Dataframe
df = df.reset_index()
df = df.pivot(index='date', columns='symbol', values='close').reset_index()
df.columns = ['date', 'IVV', 'XLG']
df = df.rename(columns = {'date':'Date'})
df = df.set_index('Date')

#Calculate the cummultive Returns and then the indexed performance with base 100
df = df.pct_change() * 100
df = round(100 * (1 + df / 100).cumprod(), 2)
df = df.fillna(100)

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = df

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column+'{0,.00}'}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Index (Base=100)')
    source_curves.append(src)
    target_curves.append(tgt)

# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('Invesco S&P 500 Top 50 ETF (XLG) vs. iShares Core S&P 500 ETF (IVV)').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph = full_graph.opts(merge_tools=False, shared_axes=False)

In [ ]:
#Save graph to Html
p = pn.panel(full_graph)
p.save('XLG_vs_IVV_Graph.html', embed = True)

# Current Top 10 Holdings of IVV and Performance

In [ ]:
#Download the data from iShares
url = 'https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund'
curr_top10_df = pd.read_csv(url, skiprows=9, delimiter=',', error_bad_lines=False)

#Filter for the Stocks in the ETF and pro-rate the weight so that it sums up to 100% (exclude Cash, etc.)
curr_top10_df = curr_top10_df[curr_top10_df['Asset Class'] == 'Equity']
curr_top10_df['Weight (%)'] = curr_top10_df['Weight (%)'] *(100 / curr_top10_df['Weight (%)'].sum())
curr_top10_df['Weight (%)'] = round(curr_top10_df['Weight (%)'], 2)

#Limit the data for the Top 10 [will sum GOOGL with GOOG]
curr_top10_df = curr_top10_df[:11]

#Keep only relevant columns
curr_top10_df = curr_top10_df[['Ticker', 'Name', 'Weight (%)']]

#Sum GOOGL and GOOG, drop GOOG, and change the name for 'Class' not to appear 
curr_top10_df.loc[4, 'Weight (%)'] = curr_top10_df.loc[4:5, 'Weight (%)'].sum()
curr_top10_df = curr_top10_df.drop(5)
curr_top10_df.loc[4, 'Name'] = 'ALPHABET INC CLASS'

#Change the Ticker of Berkshire
curr_top10_df.loc[7, 'Ticker'] = 'BRK-B'

In [ ]:
#Create a List with the Top 10 Holdings in S&P 500
top10_tickers = curr_top10_df['Ticker'].to_list()

tickers = Ticker(top10_tickers, asynchronous=True)
top10_perf_df = tickers.history(start = '2022-12-30')

#Reorganize the Dataframe
top10_perf_df = top10_perf_df.reset_index()
top10_perf_df = top10_perf_df.pivot(index='date', columns='symbol', values='close').reset_index()
top10_perf_df = top10_perf_df.set_index('date')

#Calculate the YTD Performance
top10_perf_df = pd.DataFrame(round((top10_perf_df.iloc[-1] / top10_perf_df.iloc[0] - 1) * 100, 2))
top10_perf_df = top10_perf_df.reset_index()
top10_perf_df = top10_perf_df.rename(columns = {top10_perf_df.columns[0]: 'Ticker', top10_perf_df.columns[1]: 'Return (%)'})

In [ ]:
#Merge the Top 10 Holdings performance with the Current Top 10 dataframe
curr_top10_df = curr_top10_df.merge(top10_perf_df, on = 'Ticker', how='left')

In [ ]:
#Calculate the Top 10 Total Weight
top10_weight_sum = curr_top10_df['Weight (%)'].sum()

#Calculate the Top 10 Average Return
top10_wgt_avg_return = round((curr_top10_df['Weight (%)'] * curr_top10_df['Return (%)'] / 100).sum(), 2)

#Add the Data to the Curr_top10 dataframe
top10_weight_data = final_df = pd.DataFrame([{'Ticker':'-', 'Name':'Total / Weighted Average', 'Weight (%)':top10_weight_sum, 
                                              'Return (%)': top10_wgt_avg_return}])
curr_top10_df = pd.concat([curr_top10_df, top10_weight_data], axis=0)

In [ ]:
#Create Holoviews table object for Top 10 Companies
curr_top10_df_table = hv.Table(curr_top10_df).opts(
    opts.Table(width=950, height=350, selectable = True, index_position = None, 
               title = 'Current Top 10 Companies in the S&P 500 Index'))

In [ ]:
#Save the Plots
p = pn.panel(curr_top10_df_table)
p.save('Current_SP500_Top10_Companies.html', embed = True)

# Historical Percentage on Top 10 Companies

In [ ]:
#Instantiate an object with all the tabs in IVV_EDGAR_holdings_full file
ivv_edgar_holdings = pd.read_excel('IVV_EDGAR_holdings_full.xlsx', sheet_name=None)

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
hist_top10_weight_df = pd.DataFrame(columns=['Year', 'Weight (%)'])

#Run a Loop to get the Top 10 Total Weight for each year
for year in ivv_edgar_holdings.keys():
    
    #Get the Top 10 Total Weight and Create a DataFrame with the data
    top10_year_weight = ivv_edgar_holdings[year].iloc[:10]['Weight (%)'].sum() * 100
    year_top10_df = pd.DataFrame([{'Year':year, 'Weight (%)':top10_year_weight}])

    #Concatenate the New data in the hist_top10_weight_df dataframe
    hist_top10_weight_df = pd.concat([hist_top10_weight_df, year_top10_df])

In [ ]:
#Convert the Year column to date showing the last day of the year
hist_top10_weight_df['Year'] = pd.to_datetime(hist_top10_weight_df['Year'], format='%Y') + pd.offsets.YearEnd()

#Get Current Date and Join current holdings to the Historical Top 10 Dataframe
current_date = pd.to_datetime((pd.Timestamp.today() - pd.offsets.BDay(1)).date())
current_top10_weight_df = pd.DataFrame([{'Year':current_date, 'Weight (%)':top10_weight_sum}])

#Join the Current Date data to the Historical data
hist_top10_weight_df = pd.concat([hist_top10_weight_df, current_top10_weight_df])

#Rename Columns and set Index
hist_top10_weight_df = hist_top10_weight_df.rename(columns = {'Year':'Date', 'Weight (%)':'Weight'})
hist_top10_weight_df = hist_top10_weight_df.set_index('Date')

#Sort Values
hist_top10_weight_df = hist_top10_weight_df.sort_values(by='Date')

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = hist_top10_weight_df

#Create the Graph
hover = HoverTool(tooltips=[("Date", "@Date{%F}"), ('Weight', f"@Weight")], formatters={'@Date': 'datetime'})  
hist_top10_weight_graph = hv.Curve(graph_df).opts(opts.Curve(tools=[hover])).opts(width=800, height=400, ylabel = 'Weight (%)',color = 'red', xticks=10
                                                               ).relabel('Historial Top10 Weight (%) of iShares Core S&P 500 ETF (IVV)')

In [ ]:
#Save the Plots
p = pn.panel(hist_top10_weight_graph)
p.save('Historical_Top10_Weights.html', embed = True)

# Historical Top 10 Companies

In [ ]:
#Create a New Dictionary with only the Top10 Companies [Note that the ivv_edgar_holdings has all the 500 holdings]
ivv_edgar_holdings_top10 = {}

#Iterate over the ivv_edgar_holdings dictionary
for key, df in ivv_edgar_holdings.items():
    #Take the first 10 rows of each dataframe
    limited_df = df.head(10)
    
    #Multiply the Weights by 100
    limited_df['Weight (%)'] = limited_df['Weight (%)'] * 100
    
    #Add the limited dataframe to the new dictionary
    ivv_edgar_holdings_top10[key] = limited_df

In [ ]:
#Get the list of years available
years = list(ivv_edgar_holdings_top10.keys())

In [ ]:
def load_year(Year):
    table = hv.Table(ivv_edgar_holdings_top10[Year])
    return table
    
dmap = hv.DynamicMap(load_year, kdims='Year').redim.values(Year=years)

dmap = dmap.opts(framewise=True)

In [ ]:
#Let's use the Panel library to be able to save the Table generated
p = pn.panel(dmap,  widget_location='top_left')
p.save('Historical_SP500_Top10_Companies.html', embed = True, max_states=18)